In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import pickle
import random

In [3]:
with open(r'D:\study\thesis\project\HBDM-main\datasets\ppi\ppi_index.pkl', 'rb') as f:
    value_to_index_mapping = pickle.load(f)
len(value_to_index_mapping)

18767

In [4]:
local_stringdb = 'D:/study/thesis/project/HBDM-main/nn_data/stringdb/'
# load local STRING database and names
df = pd.read_csv(local_stringdb+'9606.protein.info.v12.0.txt', sep='\t', header=0, usecols=['#string_protein_id', 'preferred_name'])
df['preferred_name'] = df['preferred_name'].str.upper()
stringId2name = df.set_index('#string_protein_id')['preferred_name'].to_dict()
name2stringId = df.set_index('preferred_name')['#string_protein_id'].to_dict()
df = pd.read_csv(local_stringdb+'9606.protein.aliases.v12.0.txt', sep='\t', header=0, usecols=['#string_protein_id', 'alias']).drop_duplicates(['alias'], keep='first')
df['alias'] = df['alias'].str.upper()
aliases2stringId = df.set_index('alias')['#string_protein_id'].to_dict()


network = pd.read_csv(local_stringdb+'9606.protein.physical.links.detailed.v12.0.txt', sep=' ', header=0).convert_dtypes().replace(0, float('nan'))

network = network[['protein1','protein2','combined_score']]

In [5]:
# Define the reversed bin edges and labels
bins = [149, 199, 299, 399, 499, 599, 699, 799, 899, 999]
labels = [9, 8, 7, 6, 5, 4, 3, 2, 1]

# Use pd.cut to categorize the data into levels
network['weight'] = pd.cut(network['combined_score'], bins=bins, labels=labels, include_lowest=True)



In [6]:
network.head(3)

,protein1,protein2,combined_score,weight
0,9606.ENSP00000000233,9606.ENSP00000257770,311,7
1,9606.ENSP00000000233,9606.ENSP00000226004,161,9
2,9606.ENSP00000000233,9606.ENSP00000434442,499,6


In [7]:
network['protein1'] = network['protein1'].str.replace('9606.ENSP', '')
network['protein2'] = network['protein2'].str.replace('9606.ENSP', '')
node_array = np.array(network[['protein1','protein2']].to_numpy(), dtype=int)
mapped_arr = np.vectorize(value_to_index_mapping.get)(node_array)
mapped_arr = mapped_arr.T
mapped_arr.shape

(2, 1477610)

In [8]:
network['node1'] = mapped_arr[0]
network['node2'] = mapped_arr[1]

In [9]:
network.head(3)

,protein1,protein2,combined_score,weight,node1,node2
0,00000000233,00000257770,311,7,0,1914
1,00000000233,00000226004,161,9,0,776
2,00000000233,00000434442,499,6,0,15878


In [10]:
# Create an empty graph
G = nx.Graph()

# Iterate over the DataFrame rows and add edges with weights to the graph
for index, row in network.iterrows():
    G.add_edge(row['node1'], row['node2'], weight=row['weight'])

In [11]:
len(G.edges)

738805

# Test

### complexs

In [12]:
complexs = {'RNA':['RPB1', 'RPB2', 'RPB3', 'RPB4', 'RPB5', 'RPB6', 'RPB7', 'RPB8', 'RPB9', 'RPB10', 'RPB11', 'RPB12', 'RPB13'],
    'protease': ['PSMA1', 'PSMA2', 'PSMA3', 'PSMB1', 'PSMB2', 'PSMB3'],
    'nuclear pore': ['NUP98', 'NUP93', 'NUP107', 'NUP133']
}

names = set(aliases2stringId.keys())
humans = set(value_to_index_mapping.keys())

complexs_id = dict()

for complex_name in complexs:
    folder_path = 'D:/study/thesis/project/HBDM-main/ppi_results/test_results/'+complex_name
    # os.mkdir(folder_path)
    group_node = []
    for gene in complexs[complex_name]:
        if gene in names:
            stringid = aliases2stringId[gene]
            stringid = int(stringid[9:])
            if stringid in humans:
                node = value_to_index_mapping[stringid]
                group_node.append(node)
    complexs_id[complex_name] = group_node

In [13]:
for complex_name in complexs_id:
    points = complexs_id[complex_name]
    for top in [100,200,300]:
        precision = []
        coverage = []
        for start_gene in points:
            test_nodes = list(set(points)-set([start_gene]))
            true_pre = []
            subdf = network[network['node1']==start_gene]
            ranked = subdf.sort_values(by='weight')
            predicted=ranked[:top]['node2'].tolist()
            for i in predicted:
                if i in test_nodes:
                    true_pre.append(i)
            precision.append(len(true_pre)/len(predicted))
            coverage.append(len(set(true_pre))/len(test_nodes))
        print('top-',top,'\t',complex_name,' precision: ', sum(precision)/len(precision))
        print(complex_name,' coverage: ', sum(coverage)/len(coverage))

top- 100 	 RNA  precision:  0.0
RNA  coverage:  0.0
top- 200 	 RNA  precision:  0.0
RNA  coverage:  0.0
top- 300 	 RNA  precision:  0.0022222222222222222
RNA  coverage:  0.06060606060606061
top- 100 	 protease  precision:  0.0
protease  coverage:  0.0
top- 200 	 protease  precision:  0.020796864242582785
protease  coverage:  0.7333333333333334
top- 300 	 protease  precision:  0.026179452348131044
protease  coverage:  1.0
top- 100 	 nuclear pore  precision:  0.0
nuclear pore  coverage:  0.0
top- 200 	 nuclear pore  precision:  0.018596125519840318
nuclear pore  coverage:  1.0
top- 300 	 nuclear pore  precision:  0.018596125519840318
nuclear pore  coverage:  1.0


### disease

In [14]:
disease_df = pd.read_csv(r'D:\study\thesis\project\HBDM-main\disease\Coronary artery disease.tsv',sep='\t')
names = set(aliases2stringId.keys())
humans = set(value_to_index_mapping.keys())

group_node = []
for gene in disease_df['Gene']:
    if gene in names:
        stringid = aliases2stringId[gene]
        stringid = int(stringid[9:])
        if stringid in humans:
            node = value_to_index_mapping[stringid]
            group_node.append(node)

In [15]:
results = []
for start in group_node:
    path_weight = nx.shortest_path_length(G, source=start, weight='weight')
    results.append(path_weight)

In [16]:

from sklearn.model_selection import KFold


for k in [5,10,50]:
    k+=1
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    for train_index, test_index in kf.split(group_node):
        train_nodes = [group_node[i] for i in train_index]
        test_nodes = [group_node[i] for i in test_index]
        precision = []
        coverage = []
        for start in train_nodes:
            position = points.index(start)
            sorted_items = sorted(results[position].items(), key=lambda item: item[1])
            sorted_keys = [item[0] for item in sorted_items]
            true_pre = []
            
            k_neiboger = sorted_keys[:k]
            for nei in k_neiboger:
                if nei in test_nodes:
                    true_pre.append(nei)
            precision.append(len(true_pre)/len(df))
            coverage.append(len(set(true_pre))/len(test_nodes))
    print('k',k-1,'precision: ', sum(precision)/len(precision))
    print('k',k-1,'coverage: ', sum(coverage)/len(coverage))

ValueError: 5613 is not in list

In [ ]:
# from sklearn.model_selection import KFold
# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# for train_index, test_index in kf.split(group_node):
#     train_nodes = [group_node[i] for i in train_index]
#     test_nodes = [group_node[i] for i in test_index]
#     true_pre = []
#     precision = []
#     coverage = []
#     subdf = df[(df['start'].isin(train_nodes))&(df['end'].isin(train_nodes))]
#     for path in subdf['path'].tolist():
#         for gene in path:
#             if gene in test_nodes:
#                 true_pre.append(gene)
#     precision.append(len(true_pre)/len(df))
#     coverage.append(len(set(true_pre))/len(test_nodes))
# print('precision: ', sum(precision)/len(precision))
# print('coverage: ', sum(coverage)/len(coverage))

precision:  0.012077294685990338
coverage:  0.2222222222222222


In [ ]:
# test_nodes = [974, 5874, 17542, 3309, 1271, 13189, 7607, 5512, 18282, 18543]
# train_nodes = list(set(group_node)-set(test_nodes))
# true_pre = []
# precision = []
# coverage = []
# subdf = df[(df['start'].isin(train_nodes))&(df['end'].isin(train_nodes))]
# for path in subdf['path'].tolist():
#     for gene in path:
#         if gene in test_nodes:
#             true_pre.append(gene)

# print('precision: ', len(true_pre)/len(df))
# print('coverage: ', len(set(true_pre))/len(test_nodes))

precision:  0.00676328502415459
coverage:  0.2
